In [1]:
import numpy as np
import pandas as pd
from PIL import Image
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout


In [2]:
def merge_images(img1, img2):
    cloud_pixel = np.array([255, 255, 255])
    mask = np.all(img1 == cloud_pixel, axis=-1)
    img1[mask] = img2[mask]
    return img1


In [3]:
# Load labels
labels = pd.read_csv("level_03/train_data_labels.csv", header=None).values.squeeze()

# Load and merge training images
merged_train_images = []
for i in range(500):
    img_path1 = f"level_03/train_data/field{i:03d}_sample0.png"
    img_path2 = f"level_03/train_data/field{i:03d}_sample1.png"
    img1 = np.asarray(Image.open(img_path1).convert('RGB'))
    img2 = np.asarray(Image.open(img_path2).convert('RGB'))
    merged_img = merge_images(img1, img2)
    merged_train_images.append(merged_img)
    
train_data = np.array(merged_train_images) / 255.0  # Normalize to [0, 1]


In [4]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(200, 200, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1)  # Regression output
])

model.compile(optimizer='adam', loss='mse')


2023-10-20 16:37:37.683612: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-10-20 16:37:37.683882: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



In [5]:
model.fit(train_data, labels, epochs=20, batch_size=20, validation_split=0.1)


2023-10-20 16:37:41.804725: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-10-20 16:37:41.974984: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Epoch 1/20
23/23 [==============================] - 4s 138ms/step - loss: 39.4713 - val_loss: 3.2570


2023-10-20 16:37:45.819554: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Epoch 2/20
23/23 [==============================] - 3s 128ms/step - loss: 2.5582 - val_loss: 1.1905
Epoch 3/20
23/23 [==============================] - 3s 126ms/step - loss: 0.6555 - val_loss: 0.4004
Epoch 4/20
23/23 [==============================] - 3s 134ms/step - loss: 0.2124 - val_loss: 0.2649
Epoch 5/20
23/23 [==============================] - 3s 127ms/step - loss: 0.1941 - val_loss: 0.2034
Epoch 6/20
23/23 [==============================] - 3s 127ms/step - loss: 0.1177 - val_loss: 0.2765
Epoch 7/20
23/23 [==============================] - 3s 126ms/step - loss: 0.0958 - val_loss: 0.1369
Epoch 8/20
23/23 [==============================] - 3s 127ms/step - loss: 0.0868 - val_loss: 0.1287
Epoch 9/20
23/23 [==============================] - 3s 126ms/step - loss: 0.0688 - val_loss: 0.1010
Epoch 10/20
23/23 [==============================] - 3s 126ms/step - loss: 0.0515 - val_loss: 0.1303
Epoch 11/20
23/23 [==============================] - 3s 128ms/step - loss: 0.0569 - val_loss: 0.142

In [8]:
merged_test_images = []
for i in range(1000):
    img_path1 = f"level_03/test_data/field{i:03d}_sample0.png"
    img_path2 = f"level_03/test_data/field{i:03d}_sample1.png"
    img1 = np.asarray(Image.open(img_path1).convert('RGB'))
    img2 = np.asarray(Image.open(img_path2).convert('RGB'))
    merged_img = merge_images(img1, img2)
    merged_test_images.append(merged_img)

test_data = np.array(merged_test_images) / 255.0  # Normalize to [0, 1]


In [9]:
predictions = model.predict(test_data).squeeze()
rounded_predictions = np.round(predictions).clip(0, 5)


2023-10-20 16:40:04.082488: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [10]:
rounded_predictions = np.round(predictions).clip(0, 5)


In [11]:
np.savetxt("predictions.csv", rounded_predictions, delimiter=",", fmt='%d')
